In [16]:
# Imports & Inits

from datetime import datetime
from datetime import timezone
import polars as pl
import os

from utils import two_point_calibration, average_bottle, plot_sensor_measurement


DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")
PICARRO_DATA_DIRECTORY = os.environ.get("PICARRO_DATA_DIRECTORY")

#------------
sensor_id = 13
#------------

df = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "download", "acropolis.parquet")) \
    .filter(pl.col("system_name") == f"tum-esm-midcost-raspi-{sensor_id}") \
    .sort(pl.col("creation_timestamp")) \
    .filter(pl.col("gmp343_filtered") > 0.0) \
    .select(pl.col("creation_timestamp"),
                pl.col("gmp343_filtered"),
                pl.col("sht45_humidity"))
    

df_p = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "input/picarro.parquet")) 

In [17]:
df.head(3).collect()

creation_timestamp,gmp343_filtered,sht45_humidity
"datetime[μs, UTC]",f64,f64
2023-08-02 14:58:22.920 UTC,553.5,48.96
2023-08-02 14:58:32.930 UTC,572.7,48.99
2023-08-02 14:58:42.940 UTC,546.4,48.61


# ICOS Calibration PICARRO

In [18]:
picarro_slope = 1.0060429925902534 
picarro_intercept = 0.09305508001614271

df_p.head(1).collect()

DATE,TIME,FRAC_DAYS_SINCE_JAN1,FRAC_HRS_SINCE_JAN1,JULIAN_DAYS,EPOCH_TIME,ALARM_STATUS,INST_STATUS,CavityPressure,CavityTemp,DasTemp,EtalonTemp,species,OutletValve,CH4,CH4_dry,CO2,CO2_dry,h2o_reported,ch4_base,ch4_pzt_std,co2_base,co2_pzt_std,wlm1_offset,wlm2_offset,datetime,__index_level_0__
str,str,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[ns],i64
"""2023-06-23""","""00:00:01.489""",173.000017,4152.000414,174.000017,1.6875e9,0,963,139.983005,44.999779,43.375,45.151695,2.0,22398.639168,1.96398,2.009492,425.117804,437.149219,2.186075,1182.223133,84.583058,1085.124996,71.320269,-0.079042,-0.047342,2023-06-23 00:00:01.489,10662630


# ICOS Calibration MC13 19.12.2023

In [19]:
# ---
# Measurement
# ---

#400
start_date = datetime(2023, 12, 19, 12, 5, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 12, 33, 0).replace(tzinfo=timezone.utc)

df_p_400 = df.sort("creation_timestamp").filter(pl.col("creation_timestamp").is_between(start_date, end_date))


#600
start_date = datetime(2023, 12, 19, 12, 34, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 13, 3, 0).replace(tzinfo=timezone.utc)

df_p_600 = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))



# ---
# Averagng
# ---

measured_values = [None, None]
true_values = [427.38, 610.95]

# 400 ppm

data = df_p_400.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
measured_values[0] = average_bottle(data)

# 600 ppm
data = df_p_600.select(pl.col("gmp343_filtered")).collect().to_series().to_list()

measured_values[1] = average_bottle(data)

# calculate slope and intercept
acropolis_slope, acropolis_intercept = two_point_calibration(measured_values, true_values)
print(f"acropolis_slope = {acropolis_slope}, acropolis_intercept = {acropolis_intercept}")

# plot
plot_sensor_measurement(df_p_400.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_p_400.collect(), col_name="sht45_humidity")
plot_sensor_measurement(df_p_600.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_p_600.collect(), col_name="sht45_humidity")


Uncut average: 451.31607142857155
Cut average: 451.3568807339451
Uncut average: 632.6536842105262
Cut average: 632.3838709677419
acropolis_slope = 1.0140476829610818, acropolis_intercept = -30.317399096798397


# Bottles on 19.12.2023

In [20]:
# bottle 41

# PICARRO
# 41		16:02	16:27	399,8
print("Starting Picarro Processing")
start_date = datetime(2023, 12, 19, 14, 56, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 15, 21, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# ACROPOLIS
print("Starting ACROPOLIS Processing")
start_date = datetime(2023, 12, 19, 15, 29, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 15, 53, 0).replace(tzinfo=timezone.utc)

df_bottle = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_bottle.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * acropolis_slope + acropolis_intercept

print(f"MC18 Average: {avg}")


# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")

plot_sensor_measurement(df_bottle.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_bottle.collect(), col_name="sht45_humidity")


Starting Picarro Processing
Uncut average: 399.83020101048874
Cut average: 399.8277599852567
Picarro Average: 402.3369712562414
Starting ACROPOLIS Processing
Uncut average: 424.94946236559156
Cut average: 425.04262295081963
MC18 Average: 400.696087866181


In [31]:
# bottle 42

# PICARRO
# 42		16:32	16:53	401.7
print("Starting Picarro Processing")
start_date = datetime(2023, 12, 19, 15, 26, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 15, 47, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# ACROPOLIS
print("Starting ACROPOLIS Processing")
start_date = datetime(2023, 12, 19, 15, 55, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 16, 15, 0).replace(tzinfo=timezone.utc)

df_bottle = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_bottle.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * acropolis_slope + acropolis_intercept

print(f"MC18 Average: {avg}")


# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")

plot_sensor_measurement(df_bottle.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_bottle.collect(), col_name="sht45_humidity")


Starting Picarro Processing
Uncut average: 401.75440536145385
Cut average: 401.75542262780124
Picarro Average: 404.2762827498513
Starting ACROPOLIS Processing
Uncut average: 426.9219178082193
Cut average: 426.8583333333332
MC18 Average: 434.650663586722


In [22]:
# bottle 43

# PICARRO
# 43		16:55	17:15	401,5
print("Starting Picarro Processing")
start_date = datetime(2023, 12, 19, 15, 49, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 16, 9, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# ACROPOLIS
print("Starting ACROPOLIS Processing")
start_date = datetime(2023, 12, 19, 16, 18, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 16, 40, 0).replace(tzinfo=timezone.utc)

df_bottle = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_bottle.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * acropolis_slope + acropolis_intercept

print(f"MC18 Average: {avg}")


# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")

plot_sensor_measurement(df_bottle.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_bottle.collect(), col_name="sht45_humidity")


Starting Picarro Processing
Uncut average: 401.57708888445774
Cut average: 401.5754515833012
Picarro Average: 404.09522414166287
Starting ACROPOLIS Processing
Uncut average: 426.4337209302325
Cut average: 426.3392857142855
MC18 Average: 402.01096573705547


In [23]:
# bottle 44

# PICARRO
# 44		17:18	17:40	401,7
print("Starting Picarro Processing")
start_date = datetime(2023, 12, 19, 16, 12, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 16, 34, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# ACROPOLIS
print("Starting ACROPOLIS Processing")
start_date = datetime(2023, 12, 19, 16, 42, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 17, 2, 0).replace(tzinfo=timezone.utc)

df_bottle = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_bottle.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * acropolis_slope + acropolis_intercept

print(f"MC18 Average: {avg}")


# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")

plot_sensor_measurement(df_bottle.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_bottle.collect(), col_name="sht45_humidity")


Starting Picarro Processing
Uncut average: 401.670906375637
Cut average: 401.6539922020857
Picarro Average: 404.17423938082476
Starting ACROPOLIS Processing
Uncut average: 426.2551282051283
Cut average: 426.2803921568629
MC18 Average: 401.9512448616097


In [40]:
# bottle 45

# PICARRO
# 45		17:41	18:02	403,6
print("Starting Picarro Processing")
start_date = datetime(2023, 12, 19, 16, 36, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 16, 56, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# ACROPOLIS
print("Starting ACROPOLIS Processing")
start_date = datetime(2023, 12, 19, 17, 5, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 17, 24, 0).replace(tzinfo=timezone.utc)

df_bottle = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_bottle.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * acropolis_slope + acropolis_intercept

print(f"MC18 Average: {avg}")


# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")

plot_sensor_measurement(df_bottle.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_bottle.collect(), col_name="sht45_humidity")


Starting Picarro Processing
Uncut average: 403.73266815821967
Cut average: 403.7371890866802
Picarro Average: 406.2700250087569
Starting ACROPOLIS Processing
Uncut average: 428.6986486486486
Cut average: 428.7937499999998
MC18 Average: 433.85130546865037


In [38]:
# bottle 46

# PICARRO
# 46		18:04	18:24	403,2
print("Starting Picarro Processing")
start_date = datetime(2023, 12, 19, 16, 59, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 17, 18, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# ACROPOLIS
print("Starting ACROPOLIS Processing")
start_date = datetime(2023, 12, 19, 17, 26, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 17, 51, 0).replace(tzinfo=timezone.utc)

df_bottle = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_bottle.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * acropolis_slope + acropolis_intercept

print(f"MC18 Average: {avg}")


# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")

plot_sensor_measurement(df_bottle.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_bottle.collect(), col_name="sht45_humidity")


Starting Picarro Processing
Uncut average: 403.31663483871057
Cut average: 403.3121405492016
Picarro Average: 405.84240790611585
Starting ACROPOLIS Processing
Uncut average: 428.0670103092782
Cut average: 427.95873015873013
MC18 Average: 434.1961820527811


In [36]:
# bottle 47

# PICARRO
# 47	20.12.2023	10:49	11:09	400,2
print("Starting Picarro Processing")
start_date = datetime(2023, 12, 20, 9, 43, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 20, 10, 3, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# ACROPOLIS
print("Starting ACROPOLIS Processing")
start_date = datetime(2023, 12, 20, 10, 10, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 20, 10, 30, 0).replace(tzinfo=timezone.utc)

df_bottle = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_bottle.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * acropolis_slope + acropolis_intercept

print(f"MC18 Average: {avg}")


# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")

plot_sensor_measurement(df_bottle.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_bottle.collect(), col_name="sht45_humidity")


Starting Picarro Processing
Uncut average: 400.2430651865782
Cut average: 400.2518400391265
Picarro Average: 402.7636140227344
Starting ACROPOLIS Processing
Uncut average: 420.8421052631578
Cut average: 420.8000000000001
MC18 Average: 437.15285239980295


In [33]:
# bottle 48

# PICARRO
# 48		11:10	11:30	401,4
print("Starting Picarro Processing")
start_date = datetime(2023, 12, 20, 10, 5, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 20, 10, 24, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# ACROPOLIS
print("Starting ACROPOLIS Processing")
start_date = datetime(2023, 12, 20, 10, 31, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 20, 10, 51, 0).replace(tzinfo=timezone.utc)

df_bottle = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_bottle.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * acropolis_slope + acropolis_intercept

print(f"MC18 Average: {avg}")


# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")

plot_sensor_measurement(df_bottle.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_bottle.collect(), col_name="sht45_humidity")


Starting Picarro Processing
Uncut average: 401.3283377840646
Cut average: 401.32542333651054
Picarro Average: 403.8436849760295
Starting ACROPOLIS Processing
Uncut average: 421.8435897435897
Cut average: 421.86274509803906
MC18 Average: 436.7139216330129


In [32]:
# bottle 49

# PICARRO  
# 49		11:31	11:51   401,3
print("Starting Picarro Processing")
start_date = datetime(2023, 12, 20, 10, 26, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 20, 10, 45, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# ACROPOLIS
print("Starting ACROPOLIS Processing")
start_date = datetime(2023, 12, 20, 10, 31, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 20, 10, 51, 0).replace(tzinfo=timezone.utc)

df_bottle = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_bottle.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * acropolis_slope + acropolis_intercept

print(f"MC18 Average: {avg}")


# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")

plot_sensor_measurement(df_bottle.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_bottle.collect(), col_name="sht45_humidity")


Starting Picarro Processing
Uncut average: 401.483531410918
Cut average: 401.4866368929442
Picarro Average: 404.00587274479017
Starting ACROPOLIS Processing
Uncut average: 421.8435897435897
Cut average: 421.86274509803906
MC18 Average: 436.7139216330129


Corrected Picarro Timeshift on Windows PC

# ICOS Calibration MC13 20.12.2023

In [29]:

# ---
# Measurement
# ---

#400
start_date = datetime(2023, 12, 20, 12, 36, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 20, 13, 6, 0).replace(tzinfo=timezone.utc)

df_p_400 = df.sort("creation_timestamp").filter(pl.col("creation_timestamp").is_between(start_date, end_date))


#600
start_date = datetime(2023, 12, 20, 13, 6, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 20, 13, 36, 0).replace(tzinfo=timezone.utc)

df_p_600 = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))



# ---
# Averagng
# ---

measured_values = [None, None]
true_values = [427.38, 610.95]

# 400 ppm

data = df_p_400.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
measured_values[0] = average_bottle(data)

# 600 ppm
data = df_p_600.select(pl.col("gmp343_filtered")).collect().to_series().to_list()

measured_values[1] = average_bottle(data)

# calculate slope and intercept
acropolis_slope, acropolis_intercept = two_point_calibration(measured_values, true_values)
print(f"acropolis_slope = {acropolis_slope}, acropolis_intercept = {acropolis_intercept}")

# plot
plot_sensor_measurement(df_p_400.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_p_400.collect(), col_name="sht45_humidity")
plot_sensor_measurement(df_p_600.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_p_600.collect(), col_name="sht45_humidity")


Uncut average: 444.54070796460184
Cut average: 444.4621621621622
Length of list to average is 0.
acropolis_slope = -0.4130160351715709, acropolis_intercept = 610.95


In [30]:
# bottle 50

# PICARRO
# 50		13:54	
print("Starting Picarro Processing")
start_date = datetime(2023, 12, 20, 12, 45, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 20, 10, 45, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# ACROPOLIS
print("Starting ACROPOLIS Processing")
start_date = datetime(2023, 12, 20, 10, 31, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 20, 10, 51, 0).replace(tzinfo=timezone.utc)

df_bottle = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_bottle.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * acropolis_slope + acropolis_intercept

print(f"MC18 Average: {avg}")


# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")

plot_sensor_measurement(df_bottle.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_bottle.collect(), col_name="sht45_humidity")


Starting Picarro Processing
Length of list to average is 0.
Picarro Average: 0.09305508001614271
Starting ACROPOLIS Processing
Uncut average: 421.8435897435897
Cut average: 421.86274509803906
MC18 Average: 436.7139216330129
